# Cohort Analysis on Online Retail Dataset - Data Preparation


## Background
xxxx

## Variables Description

* **InvoiceNo**: Invoice number. *Nominal*, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
* **StockCode**: Product (item) code. *Nominal*, a 5-digit integral number uniquely assigned to each distinct product.
* **Description**: Product (item) name. *Nominal*.
* **Quantity**: The quantities of each product (item) per transaction. *Numeric*.
* **InvoiceDate**: Invice Date and time. *Numeric*, the day and time when each transaction was generated.
* **UnitPrice**: Unit price. *Numeric*, Product price per unit in sterling.
* **CustomerID**: Customer number. *Nominal*, a 5-digit integral number uniquely assigned to each customer.
* **Country**: Country name. *Nominal*, the name of the country where each customer resides.

### Prepare dataframe

In [1]:
import pandas as pd

In [2]:
# setup dataset
df = pd.read_csv('../datasets/data.csv', encoding = "ISO-8859-1")

In [3]:
# get the first 5 records from the data
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
# get data dimension
print('Number of columns:', df.shape[0])
print('Number of rows:', df.shape[1])

Number of columns: 541909
Number of rows: 8


In [5]:
# simple info about dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 22.7+ MB


### Data Cleaning
1. Duplicated records
2. Missing values
3. Inappropriate data types
4. Cancelled items

#### 1. Duplicated records

In [6]:
print('Number of duplicated records:', df.duplicated().sum())

Number of duplicated records: 5268


In [7]:
# drop the duplicated entries
df.drop_duplicates(inplace=True)

In [8]:
# re-checking the duplicated records in dataframe
print('Number of duplicated records:', df.duplicated().sum())

Number of duplicated records: 0


#### 2. Missing values

In [9]:
# define a function for observing missing values in the dataframe
def observe_missing_values(df):
    num_of_mis_val = df.isnull().sum()
    percentage_of_mis_val = 100 * df.isnull().sum() / len(df)
    
    # create the table
    table_of_mis_val = pd.concat([num_of_mis_val, percentage_of_mis_val], axis=1)
    
    # rename the columns
    table_of_mis_val.rename(columns={0: 'Missing Values', 1: '% of Mis Val'}, inplace=True)

    # get variables with null values only
    table_of_mis_val = table_of_mis_val[table_of_mis_val['Missing Values'] != 0]
    
    return table_of_mis_val

In [10]:
# get the missing values
observe_missing_values(df)

,Missing Values,% of Mis Val
Description,1454,0.270945
CustomerID,135037,25.163377


Berdasarkan output di atas, dapat kita ketahui bahwa 25% records tidak memiliki Customer ID. Null values pada kolom tersebut tidak dapat di-impute dengan nilai lain (mean, median, atau modus). Oleh sebab itu, kita perlu menghapus records yang mengandung missing values tersebut.

In [11]:
# drop the missing values
df.dropna(inplace=True)

In [12]:
# re-check the missing values
observe_missing_values(df)

,Missing Values,% of Mis Val


#### 3. Inappropriate data types

In [13]:
# change the format of 'InvoiceDate' to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [14]:
df['InvoiceDate'].head()

0   2010-12-01 08:26:00
1   2010-12-01 08:26:00
2   2010-12-01 08:26:00
3   2010-12-01 08:26:00
4   2010-12-01 08:26:00
Name: InvoiceDate, dtype: datetime64[ns]

#### 4. Cancelled items

In [15]:
# define a function for observing cancelled items in the dataframe
def observe_cancelled_items(df):
    num_of_canceled_items = df[df['InvoiceNo'].str.contains('C')]['InvoiceNo'].count()
    percentage_of_cancelled_items = 100 * num_of_canceled_items / len(df)
    
    print('Num of cancelled items:', num_of_canceled_items)
    print("Percentage from all items: % {:.2f}".format(percentage_of_cancelled_items))

In [16]:
# get the cancelled items
observe_cancelled_items(df)

Num of cancelled items: 8872
Percentage from all items: % 2.21


In [17]:
# redefine df without the cancelled items
df = df[~df['InvoiceNo'].str.contains('C')]

In [18]:
# re-check the cancelled items
observe_cancelled_items(df)

Num of cancelled items: 0
Percentage from all items: % 0.00


### Save The Prepared Dataset

In [19]:
df.to_csv('../datasets/cleaned_data.csv')